<a href="https://colab.research.google.com/github/PremGorecki/NeuralNetwork/blob/main/03_keras/03_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import bibliotek

In [ ]:
# Przygotowanie środowiska do pracy z Tensorflow 2.0.
# Jeśli otrzymasz błąd podczas instalacji Tensorflow uruchom tę komórkę raz jeszcze.

!pip uninstall -y tensorflow
!pip install -q tensorflow==2.0.0

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

'2.0.0'

2. Załadowanie i przygotowanie danych

In [ ]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [5]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [ ]:
print(train_data[0])

In [ ]:
# jak wygląda na recenzja z tego train_data[0]
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))

In [ ]:
train_labels[:10]

In [ ]:
# kodowanie gorące jedynkowe
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension)) #pusta macierz zawierająca same zera
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

In [ ]:
test_data.shape

3. Budowa modelu bazowego

In [ ]:
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       #podajemy tu dwie metryki
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

In [13]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 161us/sample - loss: 0.4770 - accuracy: 0.8137 - binary_crossentropy: 0.4770 - val_loss: 0.3322 - val_accuracy: 0.8801 - val_binary_crossentropy: 0.3322
Epoch 2/20
25000/25000 [==============================] - 3s 124us/sample - loss: 0.2459 - accuracy: 0.9150 - binary_crossentropy: 0.2459 - val_loss: 0.2825 - val_accuracy: 0.8887 - val_binary_crossentropy: 0.2825
Epoch 3/20
25000/25000 [==============================] - 3s 124us/sample - loss: 0.1822 - accuracy: 0.9362 - binary_crossentropy: 0.1822 - val_loss: 0.2892 - val_accuracy: 0.8852 - val_binary_crossentropy: 0.2892
Epoch 4/20
25000/25000 [==============================] - 3s 123us/sample - loss: 0.1456 - accuracy: 0.9516 - binary_crossentropy: 0.1456 - val_loss: 0.3108 - val_accuracy: 0.8798 - val_binary_crossentropy: 0.3108
Epoch 5/20
25000/25000 [==============================] - 3s 121us/sample - loss: 0.1217 - acc

3. Budowa 'mniejszego' modelu

In [ ]:
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

In [15]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 3s 139us/sample - loss: 0.5760 - accuracy: 0.7603 - binary_crossentropy: 0.5760 - val_loss: 0.4641 - val_accuracy: 0.8522 - val_binary_crossentropy: 0.4641
Epoch 2/20
25000/25000 [==============================] - 3s 114us/sample - loss: 0.3723 - accuracy: 0.8843 - binary_crossentropy: 0.3723 - val_loss: 0.3524 - val_accuracy: 0.8768 - val_binary_crossentropy: 0.3524
Epoch 3/20
25000/25000 [==============================] - 3s 113us/sample - loss: 0.2787 - accuracy: 0.9099 - binary_crossentropy: 0.2787 - val_loss: 0.3067 - val_accuracy: 0.8840 - val_binary_crossentropy: 0.3067
Epoch 4/20
25000/25000 [==============================] - 3s 114us/sample - loss: 0.2291 - accuracy: 0.9243 - binary_crossentropy: 0.2291 - val_loss: 0.2895 - val_accuracy: 0.8867 - val_binary_crossentropy: 0.2895
Epoch 5/20
25000/25000 [==============================] - 3s 114us/sample - loss: 0.1971 - acc

In [ ]:
4. Budowa 'większego' modelu

In [ ]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

In [17]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 17s 698us/sample - loss: 0.3462 - accuracy: 0.8528 - binary_crossentropy: 0.3462 - val_loss: 0.2921 - val_accuracy: 0.8793 - val_binary_crossentropy: 0.2921
Epoch 2/20
25000/25000 [==============================] - 17s 676us/sample - loss: 0.1423 - accuracy: 0.9474 - binary_crossentropy: 0.1423 - val_loss: 0.3369 - val_accuracy: 0.8738 - val_binary_crossentropy: 0.3369
Epoch 3/20
25000/25000 [==============================] - 17s 676us/sample - loss: 0.0481 - accuracy: 0.9852 - binary_crossentropy: 0.0481 - val_loss: 0.4464 - val_accuracy: 0.8680 - val_binary_crossentropy: 0.4464
Epoch 4/20
25000/25000 [==============================] - 17s 671us/sample - loss: 0.0075 - accuracy: 0.9990 - binary_crossentropy: 0.0075 - val_loss: 0.5814 - val_accuracy: 0.8682 - val_binary_crossentropy: 0.5814
Epoch 5/20
25000/25000 [==============================] - 18s 710us/sample - loss: 0.0011 

In [ ]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.tail()

5. Porównanie wydajności modeli

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

6. Metody regularyzacji

In [ ]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

In [22]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 163us/sample - loss: 0.6575 - accuracy: 0.8035 - binary_crossentropy: 0.4987 - val_loss: 0.4986 - val_accuracy: 0.8764 - val_binary_crossentropy: 0.3485
Epoch 2/20
25000/25000 [==============================] - 3s 126us/sample - loss: 0.4127 - accuracy: 0.9057 - binary_crossentropy: 0.2658 - val_loss: 0.4286 - val_accuracy: 0.8880 - val_binary_crossentropy: 0.2871
Epoch 3/20
25000/25000 [==============================] - 3s 127us/sample - loss: 0.3462 - accuracy: 0.9275 - binary_crossentropy: 0.2094 - val_loss: 0.4140 - val_accuracy: 0.8872 - val_binary_crossentropy: 0.2831
Epoch 4/20
25000/25000 [==============================] - 3s 130us/sample - loss: 0.3115 - accuracy: 0.9374 - binary_crossentropy: 0.1850 - val_loss: 0.4103 - val_accuracy: 0.8830 - val_binary_crossentropy: 0.2885
Epoch 5/20
25000/25000 [==============================] - 3s 124us/sample - loss: 0.2900 - acc

In [ ]:
#prrównujemy bazowy i model ze stratą
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [ ]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

In [25]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 4s 152us/sample - loss: 0.6327 - accuracy: 0.6365 - binary_crossentropy: 0.6327 - val_loss: 0.5209 - val_accuracy: 0.8469 - val_binary_crossentropy: 0.5209
Epoch 2/20
25000/25000 [==============================] - 3s 121us/sample - loss: 0.4710 - accuracy: 0.8007 - binary_crossentropy: 0.4710 - val_loss: 0.3501 - val_accuracy: 0.8820 - val_binary_crossentropy: 0.3501
Epoch 3/20
25000/25000 [==============================] - 3s 121us/sample - loss: 0.3569 - accuracy: 0.8713 - binary_crossentropy: 0.3569 - val_loss: 0.2981 - val_accuracy: 0.8856 - val_binary_crossentropy: 0.2981
Epoch 4/20
25000/25000 [==============================] - 3s 122us/sample - loss: 0.2899 - accuracy: 0.9026 - binary_crossentropy: 0.2899 - val_loss: 0.2817 - val_accuracy: 0.8842 - val_binary_crossentropy: 0.2817
Epoch 5/20
25000/25000 [==============================] - 3s 121us/sample - loss: 0.2492 - acc

In [ ]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()